# Informed Search

### 8 Puzzle + 휴리스틱 
- 정보 있는 탐색을 위해 휴리스틱 함수와 평가 함수를 추가
- 평가함수 ```self.f()```
- 휴리스틱 ```self.h()```
- 경로비용 ```self.g()```

In [1]:
class PuzzleH:
  def __init__(self, board, goal, depth=0):
    self.board = board
    self.depth = depth
    self.goal = goal

  def get_new_board(self, i1, i2, depth):
    new_board = self.board[:]
    new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
    return PuzzleH(new_board, self.goal, depth)

  def expand(self, depth):
    result = []
    i = self.board.index(0)
    if not i in [0, 3, 6]:
      result.append(self.get_new_board(i, i-1, depth))
    if not i in [0, 1, 2]:
      result.append(self.get_new_board(i, i-3, depth))
    if not i in [2, 5, 8]:	
      result.append(self.get_new_board(i, i+1, depth))
    if not i in [6, 7, 8]:
      result.append(self.get_new_board(i, i+3, depth))
    return result


  def f(self):
    return self.h()+self.g()

  # 휴리스틱 함수 h(n): 현재 제 위치에 있지 않은 타일의 개수를 계산하여 반환
  def h(self):
    score = 0
    for i in range(9):
      if (self.board[i] != 0) and (self.board[i] != self.goal[i]):
        score += 1
    return score

  # 경로 비용 함수 g(n): 시작 노드로부터의 깊이를 반환
  def g(self):
    return self.depth

  def __eq__(self, other):
    return self.board == other.board

  def __ne__(self, other):
    return self.board != other.board

  def __lt__(self, other):
    return self.f() < other.f()

  def __gt__(self, other):
    return self.f() > other.f()

  def __str__(self):
    return str(self.board[:3]) +"\n"+\
    str(self.board[3:6]) +"\n"+\
    str(self.board[6:]) +"\n"

In [2]:
start = [2, 8, 3, 
         1, 6, 4, 
         7, 0, 5]

goal = [8, 6, 3, 
        2, 0, 4, 
        1, 7, 5]

### 최우선탐색 (Greedy Best-First Search)
- 현재 상태에서 가장 최선이라고 생각되는 것을 선택하는 방식으로 동작하는 방법, 즉, 휴리스틱 값이 작은 노드를 우선적으로 탐색하는 방법
- 우선순위 큐(Priority Queue) 대기열 활용하며, 각 노드는 튜플 사용해 (휴리스틱 값, 상태)로 표현

### 의사코드
```java
Function GBFS(initial_state, goal_state)
  p_queue ← [(h, initial_state)]
  visited ← [ ]
  while p_queue != [ ] do
    current_h, current_state ← p_queue에서 휴리스틱 h값이 가장 작은 노드 추출
    if current_state == goal_state
      return SUCCESS
    else
      current_state의 자식 노드 생성
      current_state를 visited에 추가
      if current_state의 자식 노드가 이미 p_queue나 visited에 있다면
        해당 자식 노드 건너뜀
      else
        자식 노드의 휴리스틱 값 h(n)을 계산
        남은 자식 노드들은 h값 기준으로 p_queue에 추가
  return FAIL
```

In [3]:
import heapq

def run_gbfs(initial_state, goal_state):
    pqueue = []
    visited = []

    heapq.heappush(pqueue, (initial_state.h(), initial_state))

    count = 1

    while pqueue:
        current_h, current_state = heapq.heappop(pqueue)
        print(f"count:{count}, h:{current_state.h()} \n{current_state}")
        count += 1
        
        if current_state.board == goal_state:
            return "탐색 성공"
        
        depth = current_state.depth + 1
        visited.append(current_state)

        for state in current_state.expand(depth):
            if (state in visited) and (state in [s[1] for s in pqueue]):
                continue
            else:
                heapq.heappush(pqueue, (state.h(), state))
    return "탐색 실패"

initial_state = PuzzleH(start, goal)
run_gbfs(initial_state, goal)

count:1, h:5 
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

count:2, h:4 
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]

count:3, h:3 
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]

count:4, h:2 
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]

count:5, h:1 
[8, 0, 3]
[2, 6, 4]
[1, 7, 5]

count:6, h:0 
[8, 6, 3]
[2, 0, 4]
[1, 7, 5]



'탐색 성공'

## A* 알고리즘
- 초기 노드에서 목표 노드까지의 최단 경로를 찾는 효율적인 알고리즘으로, f(n) = h(n) + g(n) 활용해 평가 함수로 활용
  1) 시작 노드에서 현재 노드까지의 경로 비용 g(n)
  2) 휴리스틱을 사용하여 현재 노드에서 목표 노드까지 이동하는 데 드는 예상 비용 h(n)
- 우선순위 큐(Priority Queue) 대기열 활용하며, 각 노드는 튜플 사용해 (f(n), 상태)로 표현

### 의사코드
```java
Function AStar(initial_state, goal_state)
  p_queue ← [f, initial_state]
  visited ← [ ]
  while p_queue != [ ] do
    current_f, current_state ← p_queue 에서 가장 평가 함수 f(n) 값이 좋은 노드
    if current_state == goal_state
      return SUCCESS
    else
      current_state의 자식 노드 생성
      current_state를 visited에 추가
      if current_state의 자식 노드가 p_queue 이나 visited 에 있으면
        해당 자식 노드 건너뜀
      else
        자식 노드의 평가 함수 값 f(n) = g(n) + h(n)을 계산
        자식 노드들을 f 값 기준으로 p_queue 에 추가
  return FAIL
```

In [4]:
import heapq

def run_astar(initial_state, goal_state):
    pqueue = []
    visited = []

    heapq.heappush(pqueue, (initial_state.f(), initial_state))

    count = 1

    while pqueue:
        current_f, current_state = heapq.heappop(pqueue)
        print(f"count:{count}, f:{current_state.f()}, h:{current_state.h()}+g:{current_state.g()} \n{current_state}")
        count += 1
        
        if current_state.board == goal_state:
            return "탐색 성공"
        
        depth = current_state.depth + 1
        visited.append(current_state)

        for state in current_state.expand(depth):
            if (state in visited) and (state in [s[1] for s in pqueue]):
                continue
            else:
                heapq.heappush(pqueue, (state.f(), state))
    return "탐색 실패"

# 초기 상태와 목표 상태는 Puzzle 클래스로 정의된다고 가정
initial_state = PuzzleH(start, goal)
run_astar(initial_state, goal)

count:1, f:5, h:5+g:0 
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

count:2, f:5, h:4+g:1 
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]

count:3, f:5, h:3+g:2 
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]

count:4, f:5, h:2+g:3 
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]

count:5, f:5, h:1+g:4 
[8, 0, 3]
[2, 6, 4]
[1, 7, 5]

count:6, f:5, h:0+g:5 
[8, 6, 3]
[2, 0, 4]
[1, 7, 5]



'탐색 성공'